# Practicumopdrachten Week 2.3 (eerste kans)

## Dependencies importeren

Kopieer in het codeblok hieronder van het vorige practicum de import-code voor de dependencies die het vaakst worden gebruikt om data in te lezen. Geef er ook de gebruikelijke aliassen aan.<br>
Zet eventuele warnings uit.

In [2]:
import pandas as pd
import sqlite3
import warnings
warnings.simplefilter('ignore')

Zet het bestand go_sales.sqlite in hetzelfde mapje als dit notebook.

## Databasetabellen inlezen

Kopieer in het codeblok hieronder van het vorige practicum de code om een connectie met het bestand go_sales.sqlite te maken.

In [2]:
sales_connection = sqlite3.connect('go_sales.sqlite')

Lees van de ingelezen go_sales-database te volgende tabellen in met behulp van "SELECT * FROM *tabel*".
- product
- product_type
- product_line
- sales_staff
- sales_branch
- retailer_site
- country
- order_header
- order_details
- target
- returned_item
- return_reason

In [3]:
tables_to_read = ['product', 'product_type', 'product_line', 'sales_staff', 'sales_branch', 
                  'retailer_site', 'country', 'order_header', 'order_details',
                  'returned_item', 'return_reason']

dataframes = {}

for table in tables_to_read:
    query = f"SELECT * FROM {table}"
    dataframes[table] = pd.read_sql_query(query, sales_connection)

product_df = dataframes['product']
product_type_df = dataframes['product_type']
product_line_df = dataframes['product_line']
sales_staff_df = dataframes['sales_staff']
sales_branch_df = dataframes['sales_branch']
retailer_site_df = dataframes['retailer_site']
country_df = dataframes['country']
order_header_df = dataframes['order_header']
order_details_df = dataframes['order_details']
returned_item_df = dataframes['returned_item']
return_reason_df = dataframes['return_reason']

print(product_df)
print(product_type_df)

    PRODUCT_NUMBER INTRODUCTION_DATE PRODUCT_TYPE_CODE PRODUCTION_COST MARGIN  \
0                1         15-2-2011                 1               4    .33   
1               10         15-2-2011                 1              10     .4   
2              100         15-2-2011                17               3     .5   
3              101        15-12-2019                18          305.54    .43   
4              102        10-12-2019                18          380.95    .51   
..             ...               ...               ...             ...    ...   
110             95         15-2-2011                16               3     .5   
111             96         15-2-2011                17           16.43    .28   
112             97          5-3-2013                17              25    .28   
113             98         15-2-2011                17               3     .5   
114             99         15-2-2011                17               2     .6   

       PRODUCT_IMAGE LANGUA

Let op! Voor alle onderstaande opdrachten mag je <u>alleen Python</u> gebruiken, <u>geen SQL!</u>

## Selecties op één tabel zonder functies

Geef een overzicht met daarin de producten en hun productiekosten waarvan de productiekosten lager dan 100 dollar en hoger dan 50 dollar ligt. (2 kolommen, 23 rijen)

In [4]:
product_df['PRODUCTION_COST'] = pd.to_numeric(product_df['PRODUCTION_COST'], errors='coerce')
producten_filtered = product_df.loc[(product_df['PRODUCTION_COST'] > 50) & (product_df['PRODUCTION_COST'] < 100), ['PRODUCT_NAME', 'PRODUCTION_COST']]

Geef een overzicht met daarin de producten en hun marge waarvan de marge lager dan 20 % of hoger dan 60 % ligt. (2 kolommen, 7 rijen) 

In [5]:
product_df['MARGIN'] = pd.to_numeric(product_df['MARGIN'], errors='coerce')
result = product_df[['PRODUCT_NAME', 'MARGIN']][(product_df['MARGIN'] < 0.20) | (product_df['MARGIN'] > 0.60)]

Geef een overzicht met daarin de landen waar met francs wordt betaald. Sorteer de uitkomst op land.  (1 kolom, 3 rijen)

In [6]:
countries_francs = country_df[['COUNTRY', 'CURRENCY_NAME']][country_df['CURRENCY_NAME'] == 'francs']
# countries_francs

Geef een overzicht met daarin de verschillende introductiedatums waarop producten met meer dan 50% marge worden geïntroduceerd (1 kolom, 7 rijen) 

In [11]:
product_df['MARGIN'] = pd.to_numeric(product_df['MARGIN'], errors='coerce')
products_high_margin = product_df[['INTRODUCTION_DATE', 'MARGIN']][product_df['MARGIN'] > 0.50]
products_high_margin_unique_dates = products_high_margin.drop_duplicates(subset='INTRODUCTION_DATE')
# products_high_margin_unique_dates

Geef een overzicht met daarin het eerste adres en de stad van verkoopafdelingen waarvan zowel het tweede adres als de regio bekend is (2 kolommen, 28 rijen)

In [9]:
verkoopafdelingen = sales_branch_df[['ADDRESS1', 'CITY']][(sales_branch_df['ADDRESS2'].notna()) & (sales_branch_df['REGION'].notna())]
# verkoopafdelingen

,ADDRESS1,CITY
10,Prol. Paseo de la Reforma No. 51,Distrito Federal
11,202-2-3 Hyakunincho,Tokyo
12,543-225 Asahi,Osaka City
13,2315 Queen's Ave,Melbourne
15,"Avenida Paulista, 333",São Paulo
21,3 Albany Court,Birmingham
27,Singelgravenplein 4,Amsterdam


Geef een overzicht met daarin de landen waar met dollars (dollars of new dollar) wordt betaald. Sorteer de uitkomst op land. (1 kolom, 4 rijen) 

Geef een overzicht met daarin beide adressen en de stad van vestigingen van klanten waarvan de postcode begint met een ‘D’ (van duitsland). Filter op vestigingen die een tweede adres hebben. (3 kolommen, 2 rijen) 

## Selecties op één tabel met functies

Geef het totaal aantal producten dat is teruggebracht (1 waarde) 

Geef het aantal regio’s waarin verkoopafdelingen gevestigd zijn. (1 waarde)

Maak 3 variabelen:
- Een met de laagste
- Een met de hoogste
- Een met de gemiddelde (afgerond op 2 decimalen)

marge van producten (3 kolommen, 1 rij) 

Geef het aantal vestigingen van klanten waarvan het 2e adres niet bekend is (1 waarde)

Geef de gemiddelde kostprijs van de verkochte producten waarop korting (unit_sale_price < unit_price) is verleend (1 waarde) 

Geef een overzicht met daarin het aantal medewerkers per medewerkersfunctie (2 kolommen, 7 rijen) 

Geef een overzicht met daarin per telefoonnummer het aantal medewerkers dat op dat telefoonnummer bereikbaar is. Toon alleen de telefoonnummer waarop meer dan 4 medewerkers bereikbaar zijn. (2 kolommen, 10 rijen) 

## Selecties op meerdere tabellen zonder functies

Geef een overzicht met daarin het eerste adres en de stad van vestigingen van klanten uit ‘Netherlands’ (2 kolommen, 20 rijen) 

Geef een overzicht met daarin de productnamen die tot het producttype ‘Eyewear’ behoren. (1 kolom, 5 rijen) 

Geef een overzicht met daarin alle unieke eerste adressen van klantvestigingen en de voornaam en achternaam van de verkopers die ‘Branch Manager’ zijn en aan deze vestigingen hebben verkocht (3 kolommen, 1 rij) 

Geef een overzicht met daarin van de verkopers hun functie en indien zij iets hebben verkocht de datum waarop de verkoop heeft plaatsgevonden. Laat alleen de verschillende namen van de posities zien van de verkopers die het woord ‘Manager’ in hun positienaam hebben staan. (2 kolommen, 9 rijen) 

Geef een overzicht met daarin de verschillende namen van producten en bijbehorende namen van producttypen van de producten waarvoor ooit meer dan 750 stuks tegelijk verkocht zijn. (2 kolommen, 9 rijen) 

Geef een overzicht met daarin de productnamen waarvan ooit meer dan 40% korting is verleend. De formule voor korting is: (unit_price - unit_sale_price) / unit_price (1 kolom, 8 rijen) 

Geef een overzicht met daarin de retourreden van producten waarvan ooit meer dan 90% van de aangeschafte hoeveelheid is teruggebracht (return_quantity/quantity). (1 kolom, 3 rijen) 

## Selecties op meerdere tabellen met functies

Geef een overzicht met daarin per producttype het aantal producten die tot dat producttype behoren. (2 kolommen, 21 rijen) 

Geef een overzicht met daarin per land het aantal vestigingen van klanten die zich in dat land bevinden. (2 kolommen, 21 rijen) 

Geef een overzicht met daarin van de producten behorend tot het producttype ‘Cooking Gear’ per productnaam de totaal verkochte hoeveelheid en de gemiddelde verkoopprijs. Sorteer de uitkomst op totaal verkochte hoeveelheid. (4 kolommen, 10 rijen) 

Geef een overzicht met daarin van de verkopers behorend tot de verkoopafdeling uit ‘Amsterdam’ de voornaam en achternaam van de verkoper en zijn laagste en hoogste verkooptarget van 2006. (4 kolommen, 4 rijen) 

Geef een overzicht met daarin per land de naam van het land, de naam van de stad waar de verkoopafdeling is gevestigd (noem de kolomnaam in het overzicht ‘verkoper’) en het aantal steden waar zich klanten bevinden in dat land (noem de kolomnaam in het overzicht ‘klanten’) (3 kolommen, 29 rijen) 

## Pythonvertalingen van SUBSELECT en UNION met o.a. for-loops

Geef een overzicht met daarin de voornaam en de achternaam van de medewerkers die nog nooit wat hebben verkocht (2 kolommen, 25 rijen) 

Geef een overzicht met daarin het aantal producten waarvan de marge lager is dan de gemiddelde marge van alle producten samen. Geef in het overzicht tevens aan wat de gemiddelde marge is van dit aantal producten waarvan de marge lager dan de gemiddelde marge van alle producten samen is. (1 kolom, 2 rijen) 

Geef een overzicht met daarin de namen van de producten die voor meer dan 500 (verkoopprijs) zijn verkocht maar nooit zijn teruggebracht. (1 kolom, 13 rijen) 

Geef een overzicht met daarin per (achternaam van) medewerker of hij/zij manager is of niet, door deze informatie toe te voegen als extra 'Ja/Nee'-kolom.<br>
Hint: gebruik een for-loop waarin je o.a. bepaalt of het woord 'Manager' in de functie (position_en) staat. (2 kolommen, 102 rijen).

Met de onderstaande code laat je Python het huidige jaar uitrekenen.

In [1]:
from datetime import date
date.today().year

2024

Met de onderstaande code selecteer je op een bepaald jaartal uit een datum.

In [2]:
from datetime import datetime

date_str = '16-8-2013'
date_format = '%d-%m-%Y'
date_obj = datetime.strptime(date_str, date_format)

date_obj.year

2013

Geef met behulp van bovenstaande hulpcode een overzicht met daarin op basis van het aantal jaar dat iemand in dienst is of een medewerker ‘kort in dienst’ (minder dan 25 jaar in dienst) of een ‘lang in dienst’ (groter gelijk dan 12 jaar in dienst) is. Geef daarbij per medewerker in een aparte kolom zowel ‘kort in dienst’ als ‘lang in dienst’ aan. Gebruik (wederom) een for-loop.<br>
(2 kolommen, 102 rijen) 